# 신경망 구현

In [8]:
# =============================================================================
# 파이토치 3장: 신경망 구현
# =============================================================================
# =============================================================================
# 실습에 필요한 라이브러리 설치 및 import
# =============================================================================
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
# import matplotlib.pyplot as plt
import numpy as np
# from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
# import seaborn as sns

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# 시드 고정 (재현 가능한 결과를 위해)
torch.manual_seed(42)
np.random.seed(42)

PyTorch version: 2.8.0+cu126
CUDA available: True


## 순전파 구현  
파이토치에서는 `nn.Module` 클래스를 활용하여 신경망을 구성하고, `forward()` 메서드를 통해 순전파를 구현한다. PyTorch에서 `nn.Module`을 상속받은 모델(model)은 내부적으로 **학습 가능한 파라미터(weight, bias)** 를 가지고 있습니다.

In [9]:
# =============================================================================
# 순전파 구현
# =============================================================================
print("\n" + "="*60)
print("파이토치에서의 순전파 구현")
print("="*60)


class SimpleNN(nn.Module):
    def __init__(self):
        # __init__은 “이 모델은 어떤 층으로 구성되어 있다”를 등록하는 부분이다.
        super().__init__()
        self.fc1 = nn.Linear(2, 3)  # 입력층(2) → 은닉층(3)
        self.fc2 = nn.Linear(3, 1)  # 은닉층(3) → 출력층(1)


    # 코드1 — 간단한 개념형 (직접 호출)
    def forward(self, x):
        # forward()는 위에서 정의한 층들이 어떻게 연결되어 계산되는지를 작성하는 곳
        x = torch.relu(self.fc1(x))   # 첫 번째 층(fc1) 이후 ReLU
        # x = torch.sigmoid(self.fc2(x))# 두 번째 층(fc2) 이후 sigmoid(출력: 0~1로 조정)
        return x

# SimpleNN이라는 간단한 신경망 생성
model = SimpleNN()
sample_input = torch.tensor([1.0, 2.0]) # 출력값
output = model(sample_input)
print(model)
print(f"SimpleNN 출력: {output}")


파이토치에서의 순전파 구현
SimpleNN(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=1, bias=True)
)
SimpleNN 출력: tensor([1.3702, 1.5487, 0.7538], grad_fn=<ReluBackward0>)


- 입력값 [1.0, 2.0]이 fc1을 거친 후 ReLU 적용한다.
- fc2를 거친 후 시그모이드를 적용하여 최종 출력한다.
- 가중치와 편향은 초기화되지 않았으므로 파이토치 기본 난수 초기값을 사용한다. 따라서 실행할 때마다 결과가 달라질 수 있다.  
- 만약 결과를 고정하고 싶다면 실행 전에 다음 코드를 추가하시오. ```torch.manual_seed(2025)```
- 직접 호출 유형은 직관적이고 짧지만,
나중에 모델 요약(print(model)) 시에는 ReLU, Sigmoid가 표시되지 않음

## 기본 신경망 클래스 설계  
2층 신경망(입력층 → 은닉층 → 출력층) 설계

In [10]:
# =============================================================================
# 기본 신경망 클래스 설계
# ======================ㅡ=======================================================
print("\n" + "="*60)
print("기본 신경망 클래스 설계")
print("="*60)

class SimpleNN_Full(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)   # 입력층 → 은닉층
        self.relu = nn.ReLU()                           # __init__에 모듈로 등록
        self.fc2 = nn.Linear(hidden_size, output_size)  # 은닉층 → 출력층
        self.sigmoid = nn.Sigmoid()

    # 코드2 — 정석적 구조형 (모듈로 등록)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
model = SimpleNN_Full(input_size=2, hidden_size=3, output_size=1)
sample_input = torch.tensor([1.0, 2.0]) # 출력값
output = model(sample_input)
print(model)


# 모델이 가진 학습 가능한 파라미터들의 형태(shape)를 출력
for param in model.parameters():
    print(param.shape)

print(f"SimpleNN_Full 출력: {output}")


기본 신경망 클래스 설계
SimpleNN_Full(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=3, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
torch.Size([3, 2])
torch.Size([3])
torch.Size([1, 3])
torch.Size([1])
SimpleNN_Full 출력: tensor([0.4412], grad_fn=<SigmoidBackward0>)


- `model.parameters() → fc1.weight, fc1.bias, fc2.weight, fc2.bias` 등을 순서대로 가져옴
- PyTorch의 `nn.Linear(in_features, out_features)`의 weight는 `(out_features, in_features)` 순서로 저장됨

### `nn.Linear(in_features, out_features)` 구조 정의

`nn.Linear(2, 3)` 의 weight는
→ **(3, 2)** 형태로 저장됩니다.
즉, `(out_features, in_features)` 순서입니다.

---

### 이유: 행렬 곱셈의 방향 때문

선형층의 수식은 기본적으로 다음과 같습니다.


$y = Wx + b$


여기서

* ( x ): 입력 벡터 (크기 = `[in_features]`)
* ( W ): 가중치 행렬
* ( b ): 편향 벡터 (크기 = `[out_features]`)
* ( y ): 출력 벡터 (크기 = `[out_features]`)

---

#### ✅ 행렬 차원 일치 조건

행렬 곱셈 ( Wx ) 이 성립하려면

* ( W ): `[out_features, in_features]`
* ( x ): `[in_features]`
  이어야 합니다.

즉, ( out $\times$ in ) × ( in ) → ( out )

---

#### 🧠 PyTorch 내부 구조

PyTorch는 이를 그대로 반영해서 `nn.Linear`를 정의합니다.

```python
self.fc1 = nn.Linear(in_features=2, out_features=3)
```

→ 내부 파라미터 구조는:

```python
fc1.weight.shape == torch.Size([3, 2])  # (out_features, in_features)
fc1.bias.shape   == torch.Size([3])
```

따라서 forward 계산은 다음과 같이 이루어집니다:

$$\text{output} = X \times W^T + b$$

즉, **PyTorch는 weight를 전치(transpose)** 해서 곱하는 구조로 처리합니다.



### 🔎 실제 예시

```python
import torch
import torch.nn as nn

fc = nn.Linear(2, 3)
x = torch.randn(1, 2)  
y = fc(x)

print(fc.weight.shape)  # torch.Size([3, 2])
print(fc.bias.shape)    # torch.Size([3])
print(y.shape)          # torch.Size([1, 3])
```



In [11]:
import torch
x = torch.randn(3, 2) # batch 크기, feature 수
print(x)
print("shape:", x.shape)

tensor([[ 0.6688, -0.7077],
        [-0.3267, -0.2788],
        [-0.4220, -1.3323]])
shape: torch.Size([3, 2])


## 학습 루프 구현  
파이토치에서는 학습 루프를 직접 정의해 신경망을 학습시킬 수 있다. 일반적으로 학습 루프에는 데이터 로딩, 순전파, 손실 계산, 역전파, 가중치 업데이트 과정이 포함된다.

### 데이터 로더 설정  
- 데이터셋을 불러오고 미니 배치 단위로 신경망에 전달하는 역할을 한다.
- `torchvision.datasets`을 활용하여 데이터를 불러올 수 있다.

- `dataset = TensorDataset(X_train, y_train)`

    * `TensorDataset`은 **입력(X)과 정답(y)을 묶어서 하나의 Dataset 객체로 만드는 클래스**입니다.
    * 이렇게 하면 나중에 `DataLoader`가 알아서 `(X, y)` 쌍을 꺼내서 배치(batch) 단위로 넘겨줄 수 있습니다.

        예를 들어 `dataset[0]`을 조회하면:

        ```python
        (X_train[0], y_train[0])
        ```

        이렇게 첫 번째 샘플과 그에 대응되는 라벨을 반환합니다.



- `dataloader = DataLoader(dataset, batch_size=32, shuffle=True)`

    * `DataLoader`는 PyTorch 학습의 핵심 구조로, **데이터를 배치 단위로 묶어서 모델에 전달**하는 역할을 합니다.

        | 매개변수            | 의미                             |
        | --------------- | ------------------------------ |
        | `dataset`       | 위에서 만든 `(X, y)` 쌍              |
        | `batch_size=32` | 한 번에 모델에 들어가는 데이터 수            |
        | `shuffle=True`  | 매 epoch마다 데이터를 섞어서 학습 순서를 무작위화 |

        예를 들어 `len(dataloader)`는 약 `100 / 32 ≈ 4`개 배치를 의미합니다.

---

실제로 동작시켜 보면:

```python
for batch_idx, (X_batch, y_batch) in enumerate(dataloader):
    print(batch_idx, X_batch.shape, y_batch.shape)
```

출력:

```
0 torch.Size([32, 10]) torch.Size([32, 1])
1 torch.Size([32, 10]) torch.Size([32, 1])
2 torch.Size([32, 10]) torch.Size([32, 1])
3 torch.Size([4, 10])  torch.Size([4, 1])  # 마지막 배치
```



In [12]:
# =============================================================================
# 데이터 로더 설정
# =============================================================================
print("\n" + "="*60)
print("데이터 로더 설정")
print("="*60)

X_train = torch.randn(100, 10)
y_train = torch.randn(100, 1)
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

print(dataset[1])


데이터 로더 설정
(tensor([-1.4181,  0.8963,  0.0499,  2.2667,  1.1790, -0.4345, -0.8140, -0.7360,
        -0.8371, -0.9224]), tensor([1.1653]))


In [13]:
for batch_idx, (X_batch, y_batch) in enumerate(dataloader):
    print(batch_idx, X_batch.shape, y_batch.shape)


0 torch.Size([32, 10]) torch.Size([32, 1])
1 torch.Size([32, 10]) torch.Size([32, 1])
2 torch.Size([32, 10]) torch.Size([32, 1])
3 torch.Size([4, 10]) torch.Size([4, 1])


## 학습 및 검증 과정 구현  
학습 과정에서는 손실 함수를 계산하고 역전파를 수행하여 가중치를 업데이트한다.  
> “입력 데이터를 넣고 → 예측값을 계산하고 → 오차(loss)를 구하고 →
오차가 작아지도록 파라미터(가중치)를 조정하는”
과정을 여러 번 반복하는 구조이다.

### 코드 전체 개요

이 코드는 **모델이 주어진 데이터(`inputs`, `targets`)를 학습**하면서
손실(loss)을 줄이는 과정을 10번(`epoch=10`) 반복하는 루프입니다.



#### 1️⃣ 손실 함수 정의

```python
criterion = nn.MSELoss()
```

* `criterion`은 **손실 함수(loss function)** 입니다.
* 여기서는 **Mean Squared Error (MSE)** 손실을 사용하고 있습니다.
  → 예측값(`outputs`)과 실제값(`targets`)의 차이 제곱 평균.
  → 주로 **회귀(regression)** 문제에 사용됩니다.

수식으로는 다음과 같습니다:
$$
\text{MSE} = \frac{1}{N} \sum_i (y_i - \hat{y_i})^2
$$
---

#### 2️⃣ 옵티마이저(Optimizer) 정의

```python
optimizer = optim.Adam(model.parameters(), lr=0.001)
```

* **Adam**은 딥러닝에서 가장 널리 쓰이는 최적화 알고리즘 중 하나입니다.
* `model.parameters()`는 모델의 모든 학습 가능한 파라미터(가중치·편향)를 가져옵니다.
* `lr=0.001`은 학습률(learning rate)로,
  → 한 번 업데이트할 때 “얼마만큼” 이동할지를 결정합니다.

즉,

> "Adam 알고리즘으로 모델의 모든 파라미터를
> 손실을 줄이는 방향으로 갱신하겠다"는 뜻입니다.

---

#### 3️⃣ 학습 루프 (10 Epoch)

```python
for epoch in range(10):
```

* **Epoch**: 데이터셋 전체를 한 번 학습시키는 주기입니다.
* 즉, 10번 반복 = 같은 데이터를 10번 돌면서 계속 개선합니다.

---

### 4️⃣ 미니배치 반복 (Dataloader)

```python
for batch in dataloader:
    inputs, targets = batch
```

* `dataloader`는 학습 데이터를 **작은 덩어리(batch)** 로 나눠서 전달해주는 PyTorch 객체입니다.
* 예를 들어, 전체 데이터가 1,000개인데 배치 크기가 100이라면,
  한 epoch에는 10개의 batch가 들어옵니다.

→ `inputs`: 모델에 들어갈 입력 데이터
→ `targets`: 정답(label)

---

#### 5️⃣ 그래디언트 초기화

```python
optimizer.zero_grad()
```

* **이전 학습 단계에서 계산된 gradient(기울기)** 가 다음 단계에 **누적** 되지 않도록 매 학습 step마다 gradient를 0으로 초기화하는 과정입니다.

* PyTorch는 gradient를 자동으로 누적(accumulate)하기 때문에,
  초기화를 안 하면 이전 단계의 값이 계속 더해져 잘못된 업데이트가 됩니다.

    ✅ 매 step마다 꼭 해야 합니다.

---

#### 6️⃣ 순전파 (Forward propagation)

```python
outputs = model(inputs)
```

* 입력 데이터(`inputs`)를 모델에 넣어 예측값(`outputs`)을 계산합니다.
* 내부적으로 `model.forward(inputs)`가 호출됩니다.

---

#### 7️⃣ 손실 계산

```python
loss = criterion(outputs, targets)
```

* 예측값(`outputs`)과 실제값(`targets`)을 비교하여 **손실(loss)** 을 계산합니다.
* 이 값이 작을수록 모델의 예측이 실제와 가깝다는 뜻입니다.

---

#### 8️⃣ 역전파 (Backward propagation)

```python
loss.backward()
```

* 손실 함수의 결과로부터 각 파라미터가 얼마나 손실에 기여했는지를 계산합니다.
* 즉, **∂Loss/∂W**를 자동으로 구합니다 (PyTorch의 `autograd` 기능).
* 이 과정을 “오차 역전파(backpropagation)”라고 부릅니다.

---

#### 9️⃣ 파라미터 업데이트

```python
optimizer.step()
```

* 역전파에서 계산된 gradient를 이용해 실제 파라미터(가중치)를 업데이트합니다.
* Adam 옵티마이저는 momentum, adaptive learning rate 등을 사용하여 안정적으로 업데이트합니다.

---

#### 🔟 학습 결과 출력

```python
print(f'Epoch {epoch+1}, Loss: {loss.item()}')
```

* `.item()`은 텐서(`torch.Tensor`) 값을 파이썬 숫자로 변환하는 함수입니다.
* 각 epoch이 끝날 때 현재 손실(loss)을 출력하여 학습이 잘 진행되는지 확인합니다.
  → Loss가 점점 줄어들면 모델이 잘 학습되고 있다는 신호입니다.

---

### ⚙️ 전체 흐름 요약

```
for each epoch:
    for each batch:
        1. 이전 gradient 초기화
        2. 입력 → 모델 → 출력 (순전파)
        3. 출력과 정답 비교 → 손실 계산
        4. 역전파로 gradient 계산
        5. optimizer로 파라미터 갱신
```

---

## 📊 시각적으로 보면

```
입력 데이터 ──▶ [Model: Forward pass] ──▶ 예측값
       │                                  │
       ▼                                  ▼
    정답(target)                    손실(Loss)
       ▲                                  │
       │─────[Backward pass]──-──┘
                 (Gradient 계산)
                       │
                Optimizer update
                       │
                 가중치(W) 갱신
```

---

### 💡 핵심 요약

| 구분        | 코드                                   | 역할            |
| --------- | ------------------------------------ | ------------- |
| 손실 함수     | `criterion = nn.MSELoss()`           | 모델의 오차 계산     |
| 옵티마이저     | `optimizer = optim.Adam(...)`        | 가중치 업데이트 알고리즘 |
| 그래디언트 초기화 | `optimizer.zero_grad()`              | 누적 방지         |
| 순전파       | `outputs = model(inputs)`            | 예측 수행         |
| 손실 계산     | `loss = criterion(outputs, targets)` | 오차 계산         |
| 역전파       | `loss.backward()`                    | 오차의 기울기 계산    |
| 업데이트      | `optimizer.step()`                   | 가중치 조정        |
| 로그        | `print(loss.item())`                 | 학습 상태 모니터링    |

---

한 문장으로 정리하자면 👇

> 이 코드는 “입력 데이터를 넣고, 예측값과 정답의 차이를 계산해,
> 그 오차를 줄이도록 모델의 가중치를 반복적으로 갱신하는”
> 딥러닝 학습의 기본 루프입니다.




In [16]:
# =============================================================================
# 학습 및 검증 과정 구현
# =============================================================================
print("\n" + "="*60)
print("학습 및 검증 과정 구현")
print("="*60)

# 피처가 10개라면 모델의 입력 차원과 샘플을 맞춰서 변경
model = SimpleNN_Full(input_size=10, hidden_size=8, output_size=1)

sample_input =  torch.randn(10)
output = model(sample_input)
print(model)

for p in model.parameters():
    print(p.shape)
print(f"SimpleNN_Full 출력: {output}")


# 손실 함수 정의(예시: 회귀)
criterion = nn.MSELoss()
# Optimizer 정의
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()  # 배치 손실 누적

    avg_loss = running_loss / len(dataloader)  # 배치 수로 나누어 평균
    print(f"Epoch {epoch+1:02d} | Train Loss: {avg_loss:.4f}")



학습 및 검증 과정 구현
SimpleNN_Full(
  (fc1): Linear(in_features=10, out_features=8, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
torch.Size([8, 10])
torch.Size([8])
torch.Size([1, 8])
torch.Size([1])
SimpleNN_Full 출력: tensor([0.5445], grad_fn=<SigmoidBackward0>)
Epoch 01 | Train Loss: 1.6238
Epoch 02 | Train Loss: 1.1984
Epoch 03 | Train Loss: 1.5135
Epoch 04 | Train Loss: 1.2917
Epoch 05 | Train Loss: 1.5495
Epoch 06 | Train Loss: 1.1840
Epoch 07 | Train Loss: 1.0770
Epoch 08 | Train Loss: 1.1155
Epoch 09 | Train Loss: 1.0387
Epoch 10 | Train Loss: 1.7053


## 체크 포인트 저장과 불러오기
### 1. 체크포인트(Checkpoint)란?

> **체크포인트란 모델 학습 중의 상태를 저장해두는 파일입니다.**

* 학습을 중단했다가 **다시 이어서 학습**할 때 사용
* 또는 학습이 완료된 모델을 **테스트나 배포용으로 불러올 때** 사용

예를 들어:

```
[에포크 50]까지 학습 → 저장(model.pth)
↓
다음날 → 불러와서 [에포크 51]부터 재학습
```

---

### 2. PyTorch에서 모델 저장 방식

PyTorch는 **모델 구조(architecture)** 와 **모델 파라미터(parameters)** 를 분리하여 관리합니다.

| 구분          | 설명                    | 예시                   |
| ----------- | --------------------- | -------------------- |
| **모델 구조**   | 신경망의 형태 (Layer 연결 구조) | `SimpleNN_Full(...)` |
| **모델 파라미터** | 학습된 가중치와 편향 값         | `model.state_dict()` |

---

### 3. 모델 저장 (Save)

```python
torch.save(model.state_dict(), 'model.pth')
```

### 🔍 해설

* `model.state_dict()` → **모델의 모든 학습 파라미터를 딕셔너리 형태로 반환**
* `'model.pth'` → 저장할 파일명 (일반적으로 `.pth` 또는 `.pt` 확장자 사용)
* `torch.save()` → 객체를 파일로 **직렬화(serialize)** 하여 저장

```python
print(model.state_dict())
```

출력 예시:

```
OrderedDict([
  ('fc1.weight', tensor([[...]])),
  ('fc1.bias', tensor([...])),
  ('fc2.weight', tensor([[...]])),
  ('fc2.bias', tensor([...]))
])
```

---

### 4. 모델 불러오기 (Load)

```python
# (1) 같은 구조의 모델 재정의
model = SimpleNN_Full(input_size=10, hidden_size=20, output_size=1)

# (2) 저장된 state_dict 불러오기
state_dict = torch.load('model.pth')

# (3) 모델에 파라미터 주입
model.load_state_dict(state_dict)
```

#### 동작 순서

| 단계 | 코드                        | 설명                |
| -- | ------------------------- | ----------------- |
| ①  | `torch.load()`            | 파일에서 딕셔너리 형태로 읽어옴 |
| ②  | `model.load_state_dict()` | 읽어온 파라미터를 모델에 복사  |
| ③  | `model.eval()` *(선택)*     | 추론(테스트) 모드로 전환    |

> ⚠️ `torch.load()`는 단순히 **파일을 읽는 함수**,
> `load_state_dict()`는 **읽은 파라미터를 모델에 적용하는 함수**입니다.

---

### 5. “전체 체크포인트” 저장하기 (모델 + 옵티마이저 + 에포크)

훈련을 재개하고 싶을 때는 모델뿐 아니라 **옵티마이저와 현재 학습 상태**도 함께 저장해야 합니다.

```python
# --- 저장 ---
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}, 'checkpoint.pth')

# --- 불러오기 ---
checkpoint = torch.load('checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
```

> ✅ 이렇게 하면 “어디서 중단됐는지” 포함한 전체 학습 상태를 그대로 복원할 수 있습니다.

---

### 6. 전체 코드 예시

```python
# =============================================================================
# 체크포인트 저장과 불러오기
# =============================================================================
print("\n" + "="*60)
print("체크 포인트 저장과 불러오기")
print("="*60)

# (1) 모델 파라미터 저장
torch.save(model.state_dict(), 'model.pth')

# (2) 같은 구조의 모델 다시 정의
model = SimpleNN_Full(input_size=10, hidden_size=20, output_size=1)

# (3) 저장된 파라미터 불러오기
model.load_state_dict(torch.load('model.pth'))

# (4) 추론 모드로 전환
model.eval()

print("✅ 모델이 성공적으로 복원되었습니다.")
```

출력 예시:

```
============================================================
체크 포인트 저장과 불러오기
============================================================
✅ 모델이 성공적으로 복원되었습니다.
```

---

### 7. 핵심 요약

| 개념                    | 역할                 | 주요 함수                          |
| --------------------- | ------------------ | ------------------------------ |
| **state_dict()**      | 모델의 파라미터 딕셔너리 반환   | `model.state_dict()`           |
| **torch.save()**      | 객체를 파일로 저장         | `torch.save(obj, path)`        |
| **torch.load()**      | 파일에서 객체를 읽음        | `torch.load(path)`             |
| **load_state_dict()** | 모델에 파라미터를 복사       | `model.load_state_dict(dict)`  |
| **checkpoint**        | 모델 + 옵티마이저 + 상태 저장 | `torch.save({...}, 'chk.pth')` |




In [18]:
# =============================================================================
# 체크 포인트 저장과 불러오기
# =============================================================================
print("\n" + "="*60)
print("체크 포인트 저장과 불러오기")
print("="*60)

torch.save(model.state_dict(), 'model.pth')
model.load_state_dict(torch.load('model.pth'))
# print(model.state_dict())


체크 포인트 저장과 불러오기


<All keys matched successfully>

In [19]:
## 실무: 훈련 재개를 위한 저장
# --- 저장 ---
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}, 'checkpoint.pth')


# --- 불러오기 ---
checkpoint = torch.load('checkpoint.pth')
print(checkpoint.keys())
epoch = checkpoint['epoch']
loss = checkpoint['loss']
# 모델과 옵티마이저 상태 복원
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss'])


In [20]:
model

SimpleNN_Full(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=20, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [21]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

## 학습 중 디버깅 및 최적화  
딥러닝 학습 도중 다양한 문제가 발생할 수 있으며, 이를 해결하기 위한 디버깅 기법과 최적화 방법이 필요하다.

### 학습이 수렴하지 않는 원인 분석
- 학습률이 너무 크거나 작으면 학습이 잘 진행되지 않을 수 있다.
- 손실 함수의 적절한 선택이 필요하다.
- 가중치 초기화가 적절하지 않을 경우 학습이 비효율적으로 진행될 수 있다.

### 손실값이 NaN이 되는 문제 해결  
- 학습률이 너무 크거나 잘못된 연산(ex. log(0))이 포함된 경우 발생할 수 있다.
- `torch.isnan(loss).any()`를 사용하여 손실값이 NaN인지 확인할 수 있다.
- 학습률 조정, 그라디언트 클리핑, GradScaler 등으로 방지

In [27]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)

In [28]:
# =============================================================================
# 손실값 NaN 처리
# =============================================================================
print("\n" + "="*60)
print("학습 중 디버깅 - NaN 처리")
print("="*60)

# 손실값이 NaN이 되는 문제 해결
def check_nan_loss(loss, optimizer):
    if torch.isnan(loss).any():
        print("Loss contains NaN values! Reducing learning rate...")
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.1 # “업데이트 보폭”을 줄여 다음 스텝에서 폭주 방지를 위해 lr 조절
        return True
    return False

# 예시 사용법
sample_loss = torch.tensor(float('nan'))
nan_detected = check_nan_loss(sample_loss, optimizer)
print(f"NaN 감지됨: {nan_detected}")


학습 중 디버깅 - NaN 처리
Loss contains NaN values! Reducing learning rate...
NaN 감지됨: True


In [23]:
sample_loss

tensor(nan)

In [26]:
optimizer.param_groups

[{'lr': 0.0001,
  'betas': (0.9, 0.999),
  'eps': 1e-08,
  'weight_decay': 0,
  'amsgrad': False,
  'maximize': False,
  'foreach': None,
  'capturable': False,
  'differentiable': False,
  'fused': None,
  'decoupled_weight_decay': False,
  'params': [Parameter containing:
   tensor([[ 0.2142,  0.1782, -0.0099,  0.3283, -0.1219, -0.0612, -0.1433,  0.0527,
             0.0649,  0.0059],
           [-0.1570, -0.1140,  0.0843, -0.2516, -0.0498, -0.0079, -0.1196,  0.1004,
             0.0136,  0.2299],
           [-0.0401, -0.2260,  0.2368,  0.0243, -0.1093, -0.2033,  0.1028, -0.3201,
             0.0983,  0.1546],
           [-0.2738, -0.2165, -0.2477,  0.2769,  0.2625,  0.0333,  0.3168, -0.1068,
            -0.3247,  0.0963],
           [ 0.1291,  0.3248, -0.0105,  0.2549,  0.1889,  0.2456, -0.3204,  0.2773,
            -0.2945,  0.2649],
           [-0.0237, -0.3050,  0.0903, -0.0728, -0.2953, -0.1002,  0.0364,  0.2682,
            -0.1090,  0.1780],
           [ 0.0312,  0.0197,  0.06

In [24]:
optimizer.param_groups[0]['lr']

0.0001

## 학습 속도 개선과 데이터 증강 기법
### 1. 학습 속도 개선
- 적절한 batch size:
    - 작을수록: 일반화 성능 ↑, 하지만 느림
    - 클수록: 속도 ↑, 메모리 사용량 ↑
    - 보통 32~128 사이에서 실험적으로 결정합니다.
- Learning Rate Scheduling:
    - epoch이 진행될수록 학습률을 점진적으로 낮춰 안정적 수렴을 유도.
    ```python
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    ```

### 2. 데이터 증강 (Data Augmentation)  
딥러닝은 “데이터가 많을수록” 일반화 성능이 향상됩니다.
하지만 실제 데이터 수집은 어렵기 때문에,
기존 데이터를 인위적으로 변형하여 다양성을 확보합니다.
> 즉, 모델이 더 많은 “경험”을 하게 만들어 과적합(Overfitting)을 방지하는 전략이에요.  

- transforms란?

`torchvision.transforms`는 PyTorch의 이미지 변환 모듈입니다.

즉, **사진을 전처리하거나 데이터 증강(Data Augmentation)** 할 때 사용하는 도구 세트예요.

예를 들어, 다음처럼 다양한 변환이 포함되어 있습니다:


| 증강 기법                      | 설명                  | PyTorch 코드                                             |
| -------------------------- | ------------------- | ------------------------------------------------------ |
| **RandomHorizontalFlip()** | 좌우 반전               | `transforms.RandomHorizontalFlip()`                    |
| **RandomRotation(10)**     | ±10도 범위 내 회전        | `transforms.RandomRotation(10)`                        |
| **ColorJitter()**          | 밝기/대비/채도 랜덤 변경      | `transforms.ColorJitter(brightness=0.2, contrast=0.2)` |
| **RandomCrop()**           | 무작위로 이미지 일부 자르기     | `transforms.RandomCrop(28)`                            |
| **RandomResizedCrop()**    | 무작위 크기·비율로 자르고 리사이즈 | `transforms.RandomResizedCrop(224)`                    |
| **GaussianBlur()**         | 블러 효과로 질감 변형        | `transforms.GaussianBlur(3)`                           |   

- `transforms.Compose([...])`의 역할  
    - Compose는 **여러 개의 변환(Transformation)** 을 “순서대로 묶어주는 함수”입니다. 쉽게 말해,
“여러 증강 단계를 하나로 묶는 pipeline(파이프라인)” 역할이에요.  
    - 아래 코드는 내부적으로 이렇게 동작합니다:
        1. 이미지 불러오기
        2. → 좌우 반전 (RandomHorizontalFlip)
        3. → 회전 (RandomRotation)
        4. → 텐서 변환 (ToTensor)
        5. → 학습 모델로 전달

        즉, Compose는 **“이 변환들을 이 순서대로 적용하라”** 는 지시문이에요.


In [29]:
# =============================================================================
# 데이터 증강 기법
# =============================================================================
print("\n" + "="*60)
print("데이터 증강 기법")
print("="*60)

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

print("데이터 증강 변환이 설정되었습니다.")


데이터 증강 기법
데이터 증강 변환이 설정되었습니다.


## PyTorch 딥러닝 기본 구조 (전체 큰 그림)  
PyTorch로 모델을 설계할 때의 기본 뼈대는 다음과 같습니다  
```python
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
```
1️⃣ 모델 정의
```python
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # (여기에 층(layer)을 정의)
    
    def forward(self, x):
        # (여기에 순전파 과정을 정의)
        return x
```
2️⃣ 모델 생성
```python
model = MyModel()
```
3️⃣ 손실 함수(loss) 정의
```python
criterion = nn.MSELoss()  # 예: 회귀 문제용 손실 함수
```
4️⃣ 옵티마이저(optimizer) 정의
```python
optimizer = optim.SGD(model.parameters(), lr=0.01)
```
5️⃣ 학습 루프(training loop)
```python
for epoch in range(epochs):
    optimizer.zero_grad()       # 이전 gradient 초기화
    outputs = model(inputs)     # 순전파(forward)
    loss = criterion(outputs, targets)  # 손실 계산
    loss.backward()             # 역전파(backpropagation)
    optimizer.step()            # 파라미터 업데이트
    running_loss += loss.item()  # 배치 손실 누적

avg_loss = running_loss / len(dataloader)  # 배치 수로 나누어 평균
print(f"Epoch {epoch+1:02d} | Train Loss: {avg_loss:.4f}")

```  

6️⃣ 모델 저장 및 불러오기  
7️⃣ 학습 중 NaN값 확인 및 학습률 조정  
8️⃣ 데이터 증강 기법 적용(이미지일 경우)  


In [31]:
# =============================================================================
# nn.Module 클래스를 이용한 신경망 구현
# =============================================================================
print("\n" + "="*60)
print("nn.Module 클래스를 이용한 신경망 구현")
print("="*60)

# 데이터셋 생성
X_train = torch.randn(100, 10)
y_train = torch.randn(100, 1)
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 신경망 모델 정의
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 모델 초기화
model = SimpleNN(input_size=10, hidden_size=20, output_size=1)
print(model)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프 구현
for epoch in range(10):
    for batch in dataloader:
        inputs, targets = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()  # 배치 손실 누적

    avg_loss = running_loss / len(dataloader)  # 배치 수로 나누어 평균
    print(f"Epoch {epoch+1:02d} | Train Loss: {avg_loss:.4f}")

# 모델 저장 및 불러오기
torch.save(model.state_dict(), 'model.pth')
model.load_state_dict(torch.load('model.pth'))

# 학습 중 NaN값 확인 및 학습률 조정
if torch.isnan(loss).any():
  print("Loss contains NaN values! Reducing learning rate...")
  for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

# 데이터 증강 기법 적용(이미지 데이터의 경우)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])


nn.Module 클래스를 이용한 신경망 구현
SimpleNN(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)
Epoch 01 | Train Loss: 16.1167
Epoch 02 | Train Loss: 17.2306
Epoch 03 | Train Loss: 18.6506
Epoch 04 | Train Loss: 19.9062
Epoch 05 | Train Loss: 21.2571
Epoch 06 | Train Loss: 22.7494
Epoch 07 | Train Loss: 23.8604
Epoch 08 | Train Loss: 25.0198
Epoch 09 | Train Loss: 26.3299
Epoch 10 | Train Loss: 27.4054
